In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from itertools import combinations
import missingno as msno
import matplotlib.ticker as mticker # NameError 해결을 위한 Ticker 임포트
import re
from typing import List, Dict, Union
from pandas.api.types import is_datetime64_any_dtype
from L_eddie_preprocessing import *

try:
    plt.rcParams['font.family'] = 'Malgun Gothic'
except:
    plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False


In [56]:
def downcast(df: pd.DataFrame, verbose: bool = True) -> pd.DataFrame:

    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        dtype_name = df[col].dtype.name
        
        if dtype_name == 'object':
            if df[col].nunique() / df[col].shape[0] < 0.5:
                df[col] = df[col].astype('category')
        
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
            
        elif dtype_name.startswith('int'):
            df[col] = pd.to_numeric(df[col], downcast='integer')
            
        elif dtype_name.startswith('float'):
            if (df[col].round() == df[col]).all():
                 df[col] = pd.to_numeric(df[col], downcast='integer')
            else:
                 df[col] = pd.to_numeric(df[col], downcast='float')
                 
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f"메모리 사용량: {start_mem:.2f} MB -> {end_mem:.2f} MB")
        print(f"감소율: {(start_mem - end_mem) / start_mem * 100:.1f}%")
        
    return df

In [57]:
train_data = pd.read_csv("data/output/rawdata/train.csv")
train_data['날짜'] = pd.to_datetime(train_data['날짜'])
train_data = downcast(train_data)

test_data = pd.read_csv("data/output/rawdata/test.csv")
test_data = downcast(test_data)

메모리 사용량: 223.99 MB -> 76.27 MB
감소율: 65.9%
메모리 사용량: 13.07 MB -> 2.82 MB
감소율: 78.4%


In [58]:
len(train_data), len(test_data)

(2935849, 214200)

In [59]:
test_data.head()

,ID,상점ID,상품ID,상점명,상품명,상품분류ID,상품분류명,월ID
0,0,5,5037,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",19,Игры - PS3,34
1,1,5,5320,"Вологда ТРЦ ""Мармелад""",ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,34
2,2,5,5233,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,34
3,3,5,5232,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,34
4,4,5,5268,"Вологда ТРЦ ""Мармелад""","Need for Speed [PS4, русская версия]",20,Игры - PS4,34


### 중복제거

In [60]:
def handle_duplicates(df):
    """
    데이터프레임의 중복 행을 찾아 출력하고, 첫 번째 행만 남기고 제거합니다.
    
    Args:
        df (pd.DataFrame): 처리할 데이터프레임
        
    Returns:
        pd.DataFrame: 중복이 제거된 데이터프레임
    """
    original_rows = len(df)
    duplicates = df[df.duplicated(keep=False)]

    if not duplicates.empty:
        print(f"총 **{len(duplicates):,}개**({duplicates.duplicated().sum():,}쌍)의 중복 행이 발견되었습니다.")
        print("\n**[중복 데이터 목록]**")
        print("```")
        print(duplicates.sort_values(by=list(df.columns)).to_string())
        print("```")
        
        print("\n중복된 행 중 첫 번째 행만 남기고 모두 제거합니다...")
        processed_df = df.drop_duplicates(keep='first')
        new_rows = len(processed_df)
        print(f"✅ **제거 완료.** 전체 행 수가 `{original_rows:,}`개에서 `{new_rows:,}`개로 변경되었습니다.")
        
        return processed_df
        
    else:
        print("✅ 중복된 행이 없습니다.")
        return df

In [61]:
train_data = handle_duplicates(train_data)

총 **12개**(6쌍)의 중복 행이 발견되었습니다.

**[중복 데이터 목록]**
```
                날짜  월ID  상점ID   상품ID    판매가  판매량                     상점명                                              상품명  상품분류ID            상품분류명
76961   2013-01-05    0    54  20130  149.0    1         Химки ТЦ "Мега"                          УЧЕНИК ЧАРОДЕЯ (регион)      40       Кино - DVD
76962   2013-01-05    0    54  20130  149.0    1         Химки ТЦ "Мега"                          УЧЕНИК ЧАРОДЕЯ (регион)      40       Кино - DVD
1435365 2014-02-23   13    50   3423  999.0    1      Тюмень ТЦ "Гудвин"  Far Cry 3 (Classics) [Xbox 360, русская версия]      23  Игры - XBOX 360
1435367 2014-02-23   13    50   3423  999.0    1      Тюмень ТЦ "Гудвин"  Far Cry 3 (Classics) [Xbox 360, русская версия]      23  Игры - XBOX 360
1496765 2014-03-23   14    21   3423  999.0    1  Москва МТРЦ "Афи Молл"  Far Cry 3 (Classics) [Xbox 360, русская версия]      23  Игры - XBOX 360
1496766 2014-03-23   14    21   3423  999.0    1  Москва МТРЦ "Афи 

### 이상치 제거 (판매가 50000이상, 판매량 1000이상)

In [62]:
# 이상치 조건에 해당하는 데이터 확인
outliers = train_data[(train_data['판매가'] >= 50000) | (train_data['판매량'] >= 1000)]

print(f"이상치 데이터 수: {len(outliers):,}개")
print("\n[이상치 데이터 샘플 (처음 10개)]")
print(outliers.head(10))

이상치 데이터 수: 5개

[이상치 데이터 샘플 (처음 10개)]
                날짜  월ID  상점ID   상품ID            판매가   판매량  \
885138  2013-09-17    8    12  11365   59200.000000     1   
1163158 2013-12-13   11    12   6066  307980.000000     1   
1488135 2014-03-20   14    25  13199   50999.000000     1   
2326930 2015-01-15   24    12  20949       4.000000  1000   
2909818 2015-10-28   33    12  11373       0.908714  2169   

                         상점명  \
885138   Интернет-магазин ЧС   
1163158  Интернет-магазин ЧС   
1488135  Москва ТРК "Атриум"   
2326930  Интернет-магазин ЧС   
2909818  Интернет-магазин ЧС   

                                                       상품명  상품분류ID  \
885138                                      Доставка (EMS)       9   
1163158                               Radmin 3  - 522 лиц.      75   
1488135            Коллекционные шахматы (Властелин Колец)      69   
2326930  Фирменный пакет майка 1С Интерес белый (34*42)...      71   
2909818               Доставка до пункта выдачи (Boxb

In [63]:
# 이상치 제거
original_rows = len(train_data)
train_data = train_data[~((train_data['판매가'] >= 6000) | (train_data['판매량'] >= 5))]

# 처리 결과 출력
print(f"이상치 제거 완료:")
print(f"- 전체 행 수: {original_rows:,}개 → {len(train_data):,}개")
print(f"- 제거된 행 수: {original_rows - len(train_data):,}개")
print(f"- 제거 비율: {(original_rows - len(train_data))/original_rows*100:.2f}%")

# 처리 후 통계 확인
print("\n[처리 후 판매가/판매량 통계]")
print(train_data[['판매가', '판매량']].describe([.01, .05, .25, .5, .75, .95, .99]))

이상치 제거 완료:
- 전체 행 수: 2,935,843개 → 2,869,752개
- 제거된 행 수: 66,091개
- 제거 비율: 2.25%

[처리 후 판매가/판매량 통계]
                판매가           판매량
count  2.869752e+06  2.869752e+06
mean   7.524252e+02  1.114773e+00
std    8.088992e+02  4.364864e-01
min   -1.000000e+00 -2.200000e+01
1%     2.800000e+01  1.000000e+00
5%     9.934000e+01  1.000000e+00
25%    2.490000e+02  1.000000e+00
50%    3.990000e+02  1.000000e+00
75%    9.900000e+02  1.000000e+00
95%    2.599000e+03  2.000000e+00
99%    3.598000e+03  3.000000e+00
max    5.999500e+03  4.000000e+00


### 상점명 유사도 비교 후 처리

In [64]:
from difflib import SequenceMatcher

def calculate_similarity(str1, str2):
    """두 문자열 간의 유사도를 계산"""
    return SequenceMatcher(None, str1, str2).ratio()

def find_similar_shop_names(df, threshold=0.8):
    """
    상점명 간의 유사도를 계산하고 유사한 쌍을 찾아냅니다.
    
    Args:
        df: 데이터프레임
        threshold: 유사도 임계값 (0~1 사이, 1에 가까울수록 더 유사함)
    """
    shop_names = df['상점명'].unique()
    similar_pairs = []
    
    for i in range(len(shop_names)):
        for j in range(i + 1, len(shop_names)):
            similarity = calculate_similarity(shop_names[i], shop_names[j])
            if similarity >= threshold:
                similar_pairs.append({
                    '상점명1': shop_names[i],
                    '상점명2': shop_names[j],
                    '유사도': similarity
                })
    
    return pd.DataFrame(similar_pairs).sort_values('유사도', ascending=False)

In [65]:
# 유사한 상점명 쌍 찾기 (유사도 70% 이상)
similar_shops = find_similar_shop_names(train_data, threshold=0.7)

if not similar_shops.empty:
    print("유사한 상점명 쌍 발견:")
    print(similar_shops.to_string(index=False))
else:
    print("유사도 70% 이상인 상점명 쌍이 없습니다.")

유사한 상점명 쌍 발견:
                                 상점명1                                            상점명2      유사도
           Жуковский ул. Чкалова 39м?                      Жуковский ул. Чкалова 39м² 0.961538
     Москва ТК "Буденовский" (пав.К7)                Москва ТК "Буденовский" (пав.А2) 0.937500
        !Якутск ТЦ "Центральный" фран                         Якутск ТЦ "Центральный" 0.884615
        !Якутск Орджоникидзе, 56 фран                         Якутск Орджоникидзе, 56 0.884615
РостовНаДону ТРК "Мегацентр Горизонт" РостовНаДону ТРК "Мегацентр Горизонт" Островной 0.880952
              Москва ТЦ "Семеновский"                          Москва ТЦ "Перловский" 0.844444
                      Химки ТЦ "Мега"                                  Омск ТЦ "Мега" 0.827586
      Москва ТЦ "МЕГА Теплый Стан" II                  Москва ТЦ "МЕГА Белая Дача II" 0.754098
                 Уфа ТК "Центральный"                         Якутск ТЦ "Центральный" 0.744186
              Казань ТЦ "ПаркХаус" I

In [66]:
# 완전히 동일한 상점들의 ID 매핑 확인
print("=== 동일 상점 ID 확인 ===")
for shop_pair in [
    ('Жуковский ул. Чкалова 39м?', 'Жуковский ул. Чкалова 39м²'),
    ('!Якутск ТЦ "Центральный" фран', 'Якутск ТЦ "Центральный"'),
    ('!Якутск Орджоникидзе, 56 фран', 'Якутск Орджоникидзе, 56')
]:
    shop1_id = train_data[train_data['상점명'] == shop_pair[0]]['상점ID'].iloc[0]
    shop2_id = train_data[train_data['상점명'] == shop_pair[1]]['상점ID'].iloc[0]
    print(f"\n{shop_pair[0]} (ID: {shop1_id})")
    print(f"{shop_pair[1]} (ID: {shop2_id})")

=== 동일 상점 ID 확인 ===

Жуковский ул. Чкалова 39м? (ID: 10)
Жуковский ул. Чкалова 39м² (ID: 11)

!Якутск ТЦ "Центральный" фран (ID: 1)
Якутск ТЦ "Центральный" (ID: 58)

!Якутск Орджоникидзе, 56 фран (ID: 0)
Якутск Орджоникидзе, 56 (ID: 57)


In [67]:
def unify_shop_ids(df):
    """
    동일 상점에 대해 상점 ID 및 상점명을 통일합니다.
    
    Args:
        df (pd.DataFrame): 상점 데이터가 포함된 데이터프레임
        
    Returns:
        pd.DataFrame: 상점 ID가 통일된 데이터프레임
    """
    id_mapping = {
        0: 57,   # Жуковский ул. Чкалова 39м? -> Жуковский ул. Чкалова 39м²
        1: 58,   # !Якутск ТЦ "Центральный" фран -> Якутск ТЦ "Центральный"
        10: 11    # !Якутск Орджоникидзе, 56 фран -> Якутск Орджоникидзе, 56
    }
    
    for old_id, new_id in id_mapping.items():
        df.loc[df['상점ID'] == old_id, '상점ID'] = new_id
        new_name = df.loc[df['상점ID'] == new_id, '상점명'].iloc[0]
        df.loc[df['상점ID'] == new_id, '상점명'] = new_name

    
    return df

In [68]:
train_data = unify_shop_ids(train_data)
test_data = unify_shop_ids(test_data)   

In [69]:
np.sort(train_data["상점ID"].unique())

array([ 2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59], dtype=int8)

### 월별 데이터 리셈플링 + 전처리를 위한 train, test 병합

In [70]:
# 1) 그룹 키
group_cols = ['월ID', '상점ID', '상품ID']

# 2) 집계
monthly_data = (
    train_data
      .groupby(group_cols)
      .agg(
          월별_판매량=('판매량', 'sum'),
          월별_판매건수=('판매량', 'size'),      # 거래 건수(행 수). 
          월별_평균_판매가=('판매가', 'mean'),
          상점명=('상점명', 'first'),
          상품명=('상품명', 'first'),
          상품분류ID=('상품분류ID', 'first'),
          상품분류명=('상품분류명', 'first')
      )
      .reset_index()
      .rename(columns={
          '월별_판매량': '월별 판매량',
          '월별_판매건수': '월별 판매건수',
          '월별_평균_판매가': '월별 평균 판매가'
      })
)

print("✅ 월별 데이터 집계 + 리네이밍 완료")
monthly_data.head(30)


✅ 월별 데이터 집계 + 리네이밍 완료


,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상점명,상품명,상품분류ID,상품분류명
0,0,2,27,1,1,2499.0,"Адыгея ТЦ ""Мега""","007 Legends [PS3, русская версия]",19,Игры - PS3
1,0,2,33,1,1,499.0,"Адыгея ТЦ ""Мега""",1+1 (BD),37,Кино - Blu-Ray
2,0,2,317,1,1,299.0,"Адыгея ТЦ ""Мега""",1С:Аудиокниги. Мединский В. Мифы о России. О р...,45,Книги - Аудиокниги 1С
3,0,2,438,1,1,299.0,"Адыгея ТЦ ""Мега""",1С:Аудиотеатр. Лучшие произведения русских пис...,45,Книги - Аудиокниги 1С
4,0,2,471,2,2,399.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8 (ред.3.0) как на ладони. Изд ...,49,Книги - Методические материалы 1С
5,0,2,481,1,1,330.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8 как на ладони. Изд 3. Гартвич...,49,Книги - Методические материалы 1С
6,0,2,482,1,1,3300.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8. Базовая версия,73,Программы - 1С:Предприятие 8
7,0,2,484,2,2,300.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8. Учебная версия. Издание 6.,73,Программы - 1С:Предприятие 8
8,0,2,491,1,1,600.0,"Адыгея ТЦ ""Мега""",1С:Деньги 8,73,Программы - 1С:Предприятие 8
9,0,2,534,2,2,399.0,"Адыгея ТЦ ""Мега""",1С:Мир компьютера Соло на клавиатуре,77,Программы - Обучающие


In [71]:
# test에 없는 컬럼을 0으로 추가하고, 순서도 train과 동일하게
test_aligned = test_data.reindex(columns=monthly_data.columns, fill_value=0)

# 행 방향으로 결합
full_data = pd.concat([monthly_data, test_aligned], ignore_index=True)


In [72]:
full_data['월별 판매량'] = full_data['월별 판매량'].clip(lower=0, upper=20)

### 상점 명을 통한 파생 변수

In [73]:

def optimize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """데이터프레임의 숫자 타입 열을 메모리에 더 효율적인 형태로 최적화합니다."""
    # This function remains the same
    for col in df.select_dtypes(include=['int', 'float']).columns:
        if pd.api.types.is_integer_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

def extract_features_from_shop_name(df: pd.DataFrame) -> pd.DataFrame:
    """'상점명' 열에서 특징을 추출하여 한국어 이름으로 된 새 열들을 생성합니다."""
    shop_name_series = df['상점명'].astype('string')
    # 이름 앞의 불필요한 '!' 문자 제거
    cleaned_shop_name = shop_name_series.str.replace(r'^\!+', '', regex=True)
    
    # 채널 유형을 식별하기 위한 불리언 마스크(boolean masks) 생성
    is_online_shop = shop_name_series.str.contains('Интернет-магазин', case=False, na=False)
    is_warehouse_online = shop_name_series.str.contains('1С-Онлайн|Цифровой склад', case=False, na=False)
    is_field_sale = shop_name_series.str.contains('Выездная Торговля', case=False, na=False)

    # .loc 접근자를 사용하여 '채널' 열 할당 (내부 값은 영어로 유지)
    df['채널'] = 'offline'
    df.loc[is_field_sale, '채널'] = 'field'
    df.loc[is_warehouse_online, '채널'] = 'warehouse_online'
    df.loc[is_online_shop, '채널'] = 'online'
    
    # 정규식을 사용하여 도시와 쇼핑몰 종류 추출
    city_pattern = r'^([А-ЯA-ZЁ][^"«(]*?)\s*(?=(МТРЦ|ТРК|ТРЦ|ТЦ|ТК|"|«|\(|ул\.|Магазин|Интернет|Цифровой|Выездная|$))'
    extracted_city = cleaned_shop_name.str.extract(city_pattern, expand=True)[0].str.strip()
    
    # 온라인, 창고, 현장 판매 채널의 경우 도시 정보가 아니므로 제거
    extracted_city = extracted_city.mask(is_online_shop | is_warehouse_online | is_field_sale)
    extracted_mall_type = cleaned_shop_name.str.extract(r'\b(МТРЦ|ТРК|ТРЦ|ТЦ|ТК)\b', expand=False).str.upper()

    # 한국어 열 이름으로 새로운 열 할당
    df['도시'] = extracted_city
    df['쇼핑몰 종류'] = extracted_mall_type
    
    # 새로 생성된 열들을 category 타입으로 변환
    channel_categories = ['offline', 'online', 'warehouse_online', 'field']
    df['채널'] = pd.Categorical(df['채널'], categories=channel_categories)
    df['도시'] = df['도시'].astype('category')
    df['쇼핑몰 종류'] = df['쇼핑몰 종류'].astype('category')
    
    return df

def impute_missing_features(df: pd.DataFrame) -> pd.DataFrame:
    """새로 생성된 특징 열에 있는 결측값(NA)을 상황에 맞게 채웁니다."""
    
    # 1. '쇼핑몰 종류' 열의 결측값(NA)을 'etc'로 채우기
    # 카테고리에 'etc'가 없는 경우 추가
    if 'etc' not in df['쇼핑몰 종류'].cat.categories:
        df['쇼핑몰 종류'] = df['쇼핑몰 종류'].cat.add_categories(['etc'])
    # 결측값을 'etc'로 채움
    df['쇼핑몰 종류'] = df['쇼핑몰 종류'].fillna('etc')

    # 2. '도시' 열의 결측값을 '채널' 정보를 기반으로 채우기
    additional_city_categories = ['Online', 'Field Sale', 'Unknown']
    existing_city_categories = df['도시'].cat.categories
    # 기존에 없는 카테고리만 추가
    categories_to_add = [cat for cat in additional_city_categories if cat not in existing_city_categories]
    if categories_to_add:
        df['도시'] = df['도시'].cat.add_categories(categories_to_add)

    # 온라인 채널(온라인, 온라인 창고)이고 도시 정보가 없는 경우 'Online'으로 채움
    is_online_channel = df['채널'].isin(['online', 'warehouse_online'])
    df.loc[df['도시'].isna() & is_online_channel, '도시'] = 'Online'
    
    # 현장 판매 채널이고 도시 정보가 없는 경우 'Field Sale'로 채움
    df.loc[df['도시'].isna() & (df['채널'] == 'field'), '도시'] = 'Field Sale'
    
    # 위 조건에 해당하지 않는 나머지 결측값은 'Unknown'으로 채움
    df['도시'] = df['도시'].fillna('Unknown')
    
    return df


In [74]:

full_data = extract_features_from_shop_name(full_data)
full_data = impute_missing_features(full_data)
full_data = optimize_dtypes(full_data)

full_data

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상점명,상품명,상품분류ID,상품분류명,채널,도시,쇼핑몰 종류
0,0,2,27,1,1,2499.0,"Адыгея ТЦ ""Мега""","007 Legends [PS3, русская версия]",19,Игры - PS3,offline,Адыгея,ТЦ
1,0,2,33,1,1,499.0,"Адыгея ТЦ ""Мега""",1+1 (BD),37,Кино - Blu-Ray,offline,Адыгея,ТЦ
2,0,2,317,1,1,299.0,"Адыгея ТЦ ""Мега""",1С:Аудиокниги. Мединский В. Мифы о России. О р...,45,Книги - Аудиокниги 1С,offline,Адыгея,ТЦ
3,0,2,438,1,1,299.0,"Адыгея ТЦ ""Мега""",1С:Аудиотеатр. Лучшие произведения русских пис...,45,Книги - Аудиокниги 1С,offline,Адыгея,ТЦ
4,0,2,471,2,2,399.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8 (ред.3.0) как на ладони. Изд ...,49,Книги - Методические материалы 1С,offline,Адыгея,ТЦ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807526,34,45,18454,0,0,0.0,"Самара ТЦ ""ПаркХаус""",СБ. Союз 55,55,Музыка - CD локального производства,offline,Самара,ТЦ
1807527,34,45,16188,0,0,0.0,"Самара ТЦ ""ПаркХаус""",Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,offline,Самара,ТЦ
1807528,34,45,15757,0,0,0.0,"Самара ТЦ ""ПаркХаус""",НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства,offline,Самара,ТЦ
1807529,34,45,19648,0,0,0.0,"Самара ТЦ ""ПаркХаус""",ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD,offline,Самара,ТЦ


### 상품 분류명을 통한 파생 변수

In [75]:
full_data[["상품대분류", "상품소분류"]] = full_data["상품분류명"].str.split(" - ", n=1, expand=True)
full_data['상품소분류'].fillna("etc", inplace=True)

C:\Users\dodor.HAEWON\AppData\Local\Temp\ipykernel_4264\3011035821.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_data['상품소분류'].fillna("etc", inplace=True)


In [76]:
full_data.head()

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상점명,상품명,상품분류ID,상품분류명,채널,도시,쇼핑몰 종류,상품대분류,상품소분류
0,0,2,27,1,1,2499.0,"Адыгея ТЦ ""Мега""","007 Legends [PS3, русская версия]",19,Игры - PS3,offline,Адыгея,ТЦ,Игры,PS3
1,0,2,33,1,1,499.0,"Адыгея ТЦ ""Мега""",1+1 (BD),37,Кино - Blu-Ray,offline,Адыгея,ТЦ,Кино,Blu-Ray
2,0,2,317,1,1,299.0,"Адыгея ТЦ ""Мега""",1С:Аудиокниги. Мединский В. Мифы о России. О р...,45,Книги - Аудиокниги 1С,offline,Адыгея,ТЦ,Книги,Аудиокниги 1С
3,0,2,438,1,1,299.0,"Адыгея ТЦ ""Мега""",1С:Аудиотеатр. Лучшие произведения русских пис...,45,Книги - Аудиокниги 1С,offline,Адыгея,ТЦ,Книги,Аудиокниги 1С
4,0,2,471,2,2,399.0,"Адыгея ТЦ ""Мега""",1С:Бухгалтерия 8 (ред.3.0) как на ладони. Изд ...,49,Книги - Методические материалы 1С,offline,Адыгея,ТЦ,Книги,Методические материалы 1С


### 필요없는 명칭 열 제거

In [77]:
full_data.drop(columns=['상점명','상품분류명', '상품명'], inplace=True)
full_data.head()

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,채널,도시,쇼핑몰 종류,상품대분류,상품소분류
0,0,2,27,1,1,2499.0,19,offline,Адыгея,ТЦ,Игры,PS3
1,0,2,33,1,1,499.0,37,offline,Адыгея,ТЦ,Кино,Blu-Ray
2,0,2,317,1,1,299.0,45,offline,Адыгея,ТЦ,Книги,Аудиокниги 1С
3,0,2,438,1,1,299.0,45,offline,Адыгея,ТЦ,Книги,Аудиокниги 1С
4,0,2,471,2,2,399.0,49,offline,Адыгея,ТЦ,Книги,Методические материалы 1С


### 신상 여부 열 추가

In [78]:
# 1) 시간열 → 월 인덱스(연*12+월)
def _to_month_index(s: pd.Series) -> pd.Series:
    if is_datetime64_any_dtype(s):
        sd = pd.to_datetime(s, errors='coerce')
        return (sd.dt.year * 12 + sd.dt.month).astype('Int32')
    if isinstance(s.dtype, pd.PeriodDtype):
        sp = s.astype('period[M]')
        return (sp.year * 12 + sp.month).astype('Int32')
    return pd.to_numeric(s, errors='coerce').astype('Int32')
    return df

# 3) 첫판매월/경과개월/신상여부(0/1) — 누적 최소(cummin) 기반, merge 없음
def add_first_sale_features_causal(
    df: pd.DataFrame,
    time_col: str,
    specs: List[Dict[str, Union[str, List[str]]]],
    int_dtype: str = 'Int16',
    add_ts: bool = False  # 필요 시 True로 바꾸면 월초 timestamp 컬럼도 추가
) -> pd.DataFrame:
    mi = _to_month_index(df[time_col])

    for spec in specs:
        keys = list(spec['keys'])
        name = str(spec['name'])

        first_idx_col = f'{name}_첫판매_월ID'
        age_col       = f'{name}_경과개월'
        new_col       = f'{name}_신상여부'
        ts_col        = f'{name}_첫판매월'

        # 재실행 안전: 기존 동일 열 제거
        drop_cols = [c for c in [first_idx_col, age_col, new_col, ts_col] if c in df.columns]
        if drop_cols:
            df.drop(columns=drop_cols, inplace=True)

        # 정렬용 뷰 만들기
        tmp = pd.DataFrame({'__mi__': mi}, index=df.index)
        for k in keys:
            tmp[k] = df[k].values

        # 키+시간 정렬 → 그룹별 누적 최소(첫 관측월 유지)
        tmp_sorted = tmp.sort_values(by=keys + ['__mi__'], kind='mergesort')
        first_sorted = tmp_sorted.groupby(keys, sort=False)['__mi__'].cummin()

        # 원래 순서로 복원
        first_idx = pd.Series(pd.NA, index=df.index, dtype='Int64')
        first_idx.loc[tmp_sorted.index] = first_sorted.values

        # 파생 열 생성
        df[first_idx_col] = first_idx.astype(int_dtype)
        df[age_col] = (mi - first_idx).astype(int_dtype)

        # 신상여부: 0/1(uint8), 결측은 0
        df[new_col] = ((df[age_col] == 0) & df[age_col].notna()).astype('uint8')

        # 옵션: 읽기 쉬운 첫판매월(월초 timestamp)
        if add_ts and (is_datetime64_any_dtype(df[time_col]) or isinstance(df[time_col].dtype, pd.PeriodDtype)):
            # month index → 월초 timestamp
            y = ((first_idx - 1) // 12).astype('Int32')
            m = (first_idx - y * 12).astype('Int8')
            df[ts_col] = pd.to_datetime(pd.DataFrame({'year': y, 'month': m, 'day': 1}), errors='coerce')

    return df



In [79]:
# 2) 첫판매/경과/신상 (예: 상품, 상품×상점 기준)
specs = [
    {'keys': ['상품ID'], 'name': '상품'},
    {'keys': ['상점ID', '상품ID'], 'name': '상품_상점'}
]
# time_col이 정수형 '월ID'든 datetime 열이든 그대로 넣으면 됩니다.
full_data = add_first_sale_features_causal(full_data, time_col='월ID', specs=specs, int_dtype='Int16', add_ts=False)

full_data[full_data["상품_첫판매_월ID"] == 32]

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,채널,도시,쇼핑몰 종류,상품대분류,상품소분류,상품_첫판매_월ID,상품_경과개월,상품_신상여부,상품_상점_첫판매_월ID,상품_상점_경과개월,상품_상점_신상여부
1533066,32,2,2791,6,4,5499.000000,20,offline,Адыгея,ТЦ,Игры,PS4,32,0,1,32,0,1
1533096,32,2,3350,6,5,2749.000000,19,offline,Адыгея,ТЦ,Игры,PS3,32,0,1,32,0,1
1533097,32,2,3351,10,5,3965.666748,20,offline,Адыгея,ТЦ,Игры,PS4,32,0,1,32,0,1
1533098,32,2,3352,8,4,2769.833252,23,offline,Адыгея,ТЦ,Игры,XBOX 360,32,0,1,32,0,1
1533099,32,2,3353,6,3,3665.666748,24,offline,Адыгея,ТЦ,Игры,XBOX ONE,32,0,1,32,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807114,34,45,2379,0,0,0.000000,31,offline,Самара,ТЦ,Игры PC,Цифра,32,2,0,34,0,1
1807187,34,45,18188,0,0,0.000000,57,offline,Самара,ТЦ,Музыка,MP3,32,2,0,34,0,1
1807192,34,45,4842,0,0,0.000000,24,offline,Самара,ТЦ,Игры,XBOX ONE,32,2,0,32,2,0
1807413,34,45,2664,0,0,0.000000,58,offline,Самара,ТЦ,Музыка,Винил,32,2,0,34,0,1


In [80]:
full_data.drop(columns=['상품_첫판매_월ID', '상품_상점_첫판매_월ID'], inplace=True)
full_data.head()

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,채널,도시,쇼핑몰 종류,상품대분류,상품소분류,상품_경과개월,상품_신상여부,상품_상점_경과개월,상품_상점_신상여부
0,0,2,27,1,1,2499.0,19,offline,Адыгея,ТЦ,Игры,PS3,0,1,0,1
1,0,2,33,1,1,499.0,37,offline,Адыгея,ТЦ,Кино,Blu-Ray,0,1,0,1
2,0,2,317,1,1,299.0,45,offline,Адыгея,ТЦ,Книги,Аудиокниги 1С,0,1,0,1
3,0,2,438,1,1,299.0,45,offline,Адыгея,ТЦ,Книги,Аудиокниги 1С,0,1,0,1
4,0,2,471,2,2,399.0,49,offline,Адыгея,ТЦ,Книги,Методические материалы 1С,0,1,0,1


### 년, 월, 분기 열 생성

In [81]:
mi = pd.to_numeric(full_data['월ID'], errors='coerce').astype('Int16')

full_data['연']   = (2013 + (mi // 12)).astype('Int16')
full_data['월']   = (1 + (mi % 12)).astype('Int8')
full_data['분기'] = ((full_data['월'] - 1) // 3 + 1).astype('Int8')

full_data.head()

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,채널,도시,쇼핑몰 종류,상품대분류,상품소분류,상품_경과개월,상품_신상여부,상품_상점_경과개월,상품_상점_신상여부,연,월,분기
0,0,2,27,1,1,2499.0,19,offline,Адыгея,ТЦ,Игры,PS3,0,1,0,1,2013,1,1
1,0,2,33,1,1,499.0,37,offline,Адыгея,ТЦ,Кино,Blu-Ray,0,1,0,1,2013,1,1
2,0,2,317,1,1,299.0,45,offline,Адыгея,ТЦ,Книги,Аудиокниги 1С,0,1,0,1,2013,1,1
3,0,2,438,1,1,299.0,45,offline,Адыгея,ТЦ,Книги,Аудиокниги 1С,0,1,0,1,2013,1,1
4,0,2,471,2,2,399.0,49,offline,Адыгея,ТЦ,Книги,Методические материалы 1С,0,1,0,1,2013,1,1


### 다양한 파생 시차피처 추가

In [82]:
# 원래는 가능 상점 + 상품 ID 조합을 본이후 시차피처들을 적용하는것이 맞음

In [83]:
import pandas as pd
from collections import defaultdict
import gc
from typing import List, Dict, Any, Tuple, Union

def add_time_features_by_spec_upgraded(
    df: pd.DataFrame,
    time_col: str,
    base_specs: List[Dict[str, Any]],
    *,
    add_time_diff: bool = False,
    leakage_shift: int = 1,
    drop_na: bool = False,
    presorted: bool = False,
) -> Tuple[pd.DataFrame, Dict[str, List[Tuple]]]:
    """
    선언형 스펙으로 시계열 파생피처 생성 (V2.2: Rolling 인덱스 에러 최종 수정)

    - `agg` 스펙 유무에 따라 로직을 분리하여 데이터 유실 문제를 원천 차단합니다.
    - rolling 피처 계산 시 발생하는 IndexError를 해결했습니다.
    """
    df_out = df.copy()
    if not pd.api.types.is_datetime64_any_dtype(df_out[time_col]):
        df_out[time_col] = pd.to_datetime(df_out[time_col], errors="coerce")

    insufficient: Dict[str, List[Tuple]] = {}

    def _calculate_features(
        work_df: pd.DataFrame,
        specs_in_group: List[Dict[str, Any]],
        grp_cols: List[str],
        agg_source_col: str,
    ) -> pd.DataFrame:
        
        all_new_cols = []
        
        for spec in specs_in_group:
            name = spec.get("name") or f"{spec['source_col']}_spec"
            ops = spec.get("ops", {}) or {}
            spec_min_periods = spec.get("min_periods", "full")

            effective_grp_cols = grp_cols
            use_dummy = not grp_cols
            if use_dummy:
                dummy = "__all__"
                work_df[dummy] = 1
                effective_grp_cols = [dummy]

            req_n = _required_points(ops, leakage_shift)
            sizes = work_df.groupby(effective_grp_cols, dropna=False, observed=True).size()
            bad = sizes[sizes < req_n].index.tolist()
            insufficient[name] = [tuple(x if isinstance(x, tuple) else (x,)) for x in bad]
            
            g = work_df.groupby(effective_grp_cols, group_keys=False, dropna=False, observed=True)
            
            lag_list = sorted(set(ops.get("lag", []) or []))
            if lag_list:
                s = g[agg_source_col]
                for lag in lag_list:
                    coln = f"{name}_lag{lag}"
                    work_df[coln] = s.shift(lag)
                    all_new_cols.append(coln)

            s_shifted = g[agg_source_col].shift(leakage_shift)
            for key, func in (("rolling_mean", "mean"), ("rolling_std", "std"), 
                             ("rolling_min", "min"), ("rolling_max", "max")):
                wins = ops.get(key, []) or []
                for w in wins:
                    mp = _min_periods(w, spec_min_periods)
                    coln = f"{name}_{key.replace('rolling_', '')}{w}"
                    rolled = s_shifted.rolling(w, min_periods=mp).agg(func)
                    
                    # 💡 [버그 수정] 불필요한 인덱스 변환을 제거하고 직접 할당합니다.
                    # rolled Series는 이미 work_df와 인덱스가 정렬되어 있습니다.
                    work_df[coln] = rolled
                    all_new_cols.append(coln)

            for op_key, op_func_str in (("diff", "diff"), ("pct_change", "pct_change")):
                periods = ops.get(op_key, []) or []
                for k in periods:
                    coln = f"{name}_{op_key}{k}"
                    work_df[coln] = g[agg_source_col].transform(lambda s: getattr(s, op_func_str)(k).shift(leakage_shift))
                    all_new_cols.append(coln)

            for span in ops.get("ewm_span", []) or []:
                coln = f"{name}_ema{span}"
                work_df[coln] = g[agg_source_col].transform(lambda s: s.shift(leakage_shift).ewm(span=span, adjust=False).mean())
                all_new_cols.append(coln)
            
            dfl: Union[bool, List[Tuple[int, int]]] = ops.get("diff_from_lags", False)
            if dfl and lag_list:
                pairs = []
                if isinstance(dfl, bool) and len(lag_list) >= 2:
                    pairs = list(zip(lag_list[:-1], lag_list[1:]))
                elif isinstance(dfl, list):
                    pairs = dfl
                
                for a, b in pairs:
                    ca, cb = f"{name}_lag{a}", f"{name}_lag{b}"
                    if ca in work_df.columns and cb in work_df.columns:
                        coln = f"{name}_diff_lag{a}_{b}"
                        work_df[coln] = work_df[ca] - work_df[cb]
                        all_new_cols.append(coln)
            
            yoy_cfg = ops.get("yoy_from_lags", False)
            if yoy_cfg and lag_list:
                yoy_pairs, min_months = [], 0
                if isinstance(yoy_cfg, bool):
                    if {1, 13}.issubset(set(lag_list)): yoy_pairs = [(1, 13)]
                elif isinstance(yoy_cfg, list):
                    yoy_pairs = yoy_cfg
                elif isinstance(yoy_cfg, dict):
                    yoy_pairs = yoy_cfg.get("pairs", [])
                    min_months = int(yoy_cfg.get("min_months", 0))

                if yoy_pairs:
                    grp_counts = None
                    if min_months > 0:
                        grp_counts = g[agg_source_col].transform("size")

                    for (a, b) in yoy_pairs:
                        ca, cb = f"{name}_lag{a}", f"{name}_lag{b}"
                        if ca in work_df.columns and cb in work_df.columns:
                            coln = f"{name}_yoy_m1" if (a==1 and b==13) else f"{name}_yoy_lag{a}_{b}"
                            denom = work_df[cb]
                            yoy = (work_df[ca] / denom - 1).where(denom.notna() & (denom != 0))
                            if grp_counts is not None:
                                yoy = yoy.where(grp_counts >= min_months)
                            work_df[coln] = yoy
                            all_new_cols.append(coln)

            if use_dummy:
                work_df.drop(columns=[dummy], inplace=True)
                
        # work_df에서 원본에 없던 새로운 컬럼들만 반환
        original_cols = set(specs_in_group[0].get("group_cols", [])) | {agg_source_col, time_col}
        final_new_cols = [c for c in work_df.columns if c not in original_cols and c in all_new_cols]
        return work_df[list(dict.fromkeys(final_new_cols))]

    def _required_points(ops: Dict[str, Any], shift: int) -> int:
        cand = [1 + shift]
        if not ops: return max(cand)
        if ops.get("lag"): cand.append(max(ops["lag"]) + shift)
        for k in ("rolling_mean", "rolling_std", "rolling_min", "rolling_max"):
            if ops.get(k): cand.append(max(ops[k]) + shift)
        if ops.get("diff"): cand.append(max(ops["diff"]) + shift)
        if ops.get("pct_change"): cand.append(max(ops["pct_change"]) + shift)
        return max(cand)

    def _min_periods(window: int, spec_min_periods: Any) -> int:
        if spec_min_periods in (None, "full"): return window
        try:
            v = int(spec_min_periods)
            return max(1, min(v, window))
        except Exception: return window

    spec_groups = defaultdict(list)
    for spec in base_specs:
        grp_cols = tuple(sorted(spec.get("group_cols", [])))
        agg = spec.get("agg")
        source_col = spec["source_col"]
        group_key = (grp_cols, agg, source_col)
        spec_groups[group_key].append(spec)

    for (grp_cols_tuple, agg, source_col), specs_in_group in spec_groups.items():
        grp_cols = list(grp_cols_tuple)
        
        if agg is None:
            print(f"🔄 Granular 피처 생성 중 (그룹: {grp_cols}, 소스: {source_col})...")
            base_cols = list(dict.fromkeys(grp_cols + [time_col, source_col]))
            work_df = df_out[base_cols].copy()
            if not presorted:
                work_df.sort_values(grp_cols + [time_col], inplace=True)
            
            new_features_df = _calculate_features(work_df, specs_in_group, grp_cols, source_col)
            df_out = pd.concat([df_out, new_features_df], axis=1)

        else:
            print(f"🔄 Aggregated 피처 생성 중 (그룹: {grp_cols}, 소스: {source_col}, 집계: {agg})...")
            merge_keys = grp_cols + [time_col]
            
            agg_df = df.groupby(merge_keys, as_index=False, observed=True).agg(
                **{f"__agg_source": (source_col, agg)}
            )
            agg_df.sort_values(merge_keys, inplace=True)

            agg_features_df = _calculate_features(agg_df.copy(), specs_in_group, grp_cols, "__agg_source")
            
            agg_df_with_features = pd.concat([agg_df[merge_keys], agg_features_df], axis=1)

            df_out = df_out.merge(
                agg_df_with_features,
                on=merge_keys,
                how="left"
            )
            
        gc.collect()

    if drop_na:
        original_cols = set(df.columns)
        created_cols = [c for c in df_out.columns if c not in original_cols]
        if created_cols:
            df_out.dropna(subset=created_cols, inplace=True)

    return df_out, insufficient

In [ ]:
df = full_data.copy()

# 0) 월 날짜열 생성 + (성능을 위해) 사전 정렬/카테고리화
df['월날짜'] = pd.to_datetime(dict(year=df['연'], month=df['월'], day=1))
df.sort_values(['상점ID','상품ID','월날짜'], inplace=True)

display(df.head())

for col in ['도시','쇼핑몰 종류','채널','상품대분류','상품소분류']:
    if col in df.columns and df[col].dtype == 'object':
        df[col] = df[col].astype('category')


# 2) 스펙 구성
base_specs = []


base_specs.append({
    "source_col": "월별 판매량",
    "name": "월별_판매량",
    "group_cols": ["상점ID","상품ID"],
    "agg": None,                     # ← 평균으로 변경
    "ops": {
        "lag": [1, 2, 3],
        "rolling_mean": [3],           # 3개월 이동평균(현재값 제외; t에서 t-1..t-3)
        "diff_from_lags": True         # 연속 lag 간 변화량: (lag1-lag2), (lag2-lag3)
    },
    "min_periods": "full"
})

base_specs.append({
    "source_col": "월별 판매건수",
    "name": "월별_판매건수",
    "group_cols": ["상점ID","상품ID"],
    "agg": None,                     
    "ops": {
        "lag": [1, 2, 3],
    },
    "min_periods": "full"
})
base_specs.append({
    "source_col": "월별 평균 판매가",
    "name": "월별_평균_판매가",
    "group_cols": ["상점ID","상품ID"],
    "agg": None,                     
    "ops": {
        "lag": [1, 2, 3],
    },
    "min_periods": "full"
})

base_specs.append({
    "source_col": "월별 평균 판매가",
    "name": "월별_평균_판매가",
    "group_cols": ["상점ID","상품ID"],
    "agg": None,
    "ops": {
        "lag": [1, 2, 3],
    },
    "min_periods": "full"
})

base_specs.append({
    "source_col": "월별 판매량",
    "name": "상품ID_월별_평균_판매량",
    "group_cols": ["상품ID"],
    "agg": "mean",                     # ← 평균으로 변경
    "ops": {
        "lag": [1, 2, 3],
    },
    "min_periods": "full"
})

base_specs.append({
    "source_col": "월별 판매량",
    "name": "상품ID_도시_월별_평균_판매량",
    "group_cols": ["상품ID", "도시"],
    "agg": "mean",                     # ← 평균으로 변경
    "ops": {
        "lag": [1, 2, 3],
    },
    "min_periods": "full"
})

base_specs.append({
    "source_col": "월별 판매량",
    "name": "상품ID_상품분류ID_월별_평균_판매량",
    "group_cols": ["상품ID", "상품분류ID"],
    "agg": "mean",                     # ← 평균으로 변경
    "ops": {
        "lag": [1, 2, 3],
    },
    "min_periods": "full"
})

base_specs.append({
    "source_col": "월별 판매량",
    "name": "상품ID_쇼핑몰_종류_월별_평균_판매량",
    "group_cols": ["상품ID", "쇼핑몰 종류"],
    "agg": "mean",                     # ← 평균으로 변경
    "ops": {
        "lag": [1, 2, 3],
    },
    "min_periods": "full"
})

# 3) 피처 생성 (고속 함수 사용)
df_feat, insufficient = add_time_features_by_spec_upgraded(
    df,
    time_col="월날짜",
    base_specs=base_specs,
    leakage_shift=1,        # 누수 방지 (t에서 최대 t-1까지만 사용)
    add_time_diff=False,    # 필요시 True로 각 스펙별 시간 간격(시간)을 추가
    drop_na=False,
    presorted=True          # 이미 정렬했으니 True → 속도↑
)


,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,채널,도시,쇼핑몰 종류,상품대분류,상품소분류,상품_경과개월,상품_신상여부,상품_상점_경과개월,상품_상점_신상여부,연,월,분기,월날짜
0,0,2,27,1,1,2499.0,19,offline,Адыгея,ТЦ,Игры,PS3,0,1,0,1,2013,1,1,2013-01-01
920401,17,2,27,1,1,498.0,19,offline,Адыгея,ТЦ,Игры,PS3,17,0,17,0,2014,6,2,2014-06-01
122205,2,2,30,1,1,359.0,40,offline,Адыгея,ТЦ,Кино,DVD,1,0,0,1,2013,3,1,2013-03-01
292915,5,2,30,1,1,399.0,40,offline,Адыгея,ТЦ,Кино,DVD,4,0,3,0,2013,6,2,2013-06-01
830746,15,2,30,1,1,169.0,40,offline,Адыгея,ТЦ,Кино,DVD,14,0,13,0,2014,4,2,2014-04-01


🔄 Granular 피처 생성 중 (그룹: ['상점ID', '상품ID'], 소스: 월별 판매량)...
🔄 Granular 피처 생성 중 (그룹: ['상점ID', '상품ID'], 소스: 월별 판매건수)...
🔄 Granular 피처 생성 중 (그룹: ['상점ID', '상품ID'], 소스: 월별 평균 판매가)...
🔄 Aggregated 피처 생성 중 (그룹: ['상품ID'], 소스: 월별 판매량, 집계: mean)...
🔄 Aggregated 피처 생성 중 (그룹: ['도시', '상품ID'], 소스: 월별 판매량, 집계: mean)...
🔄 Aggregated 피처 생성 중 (그룹: ['상품ID', '상품분류ID'], 소스: 월별 판매량, 집계: mean)...
🔄 Aggregated 피처 생성 중 (그룹: ['상품ID', '쇼핑몰 종류'], 소스: 월별 판매량, 집계: mean)...


In [85]:
df_feat.head(25)

,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,채널,도시,쇼핑몰 종류,...,상품ID_월별_평균_판매량_lag3,상품ID_도시_월별_평균_판매량_lag1,상품ID_도시_월별_평균_판매량_lag2,상품ID_도시_월별_평균_판매량_lag3,상품ID_상품분류ID_월별_평균_판매량_lag1,상품ID_상품분류ID_월별_평균_판매량_lag2,상품ID_상품분류ID_월별_평균_판매량_lag3,상품ID_쇼핑몰_종류_월별_평균_판매량_lag1,상품ID_쇼핑몰_종류_월별_평균_판매량_lag2,상품ID_쇼핑몰_종류_월별_평균_판매량_lag3
0,0,2,27,1,1,2499.000000,19,offline,Адыгея,ТЦ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17,2,27,1,1,498.000000,19,offline,Адыгея,ТЦ,...,1.000000,1.0,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,2,2,30,1,1,359.000000,40,offline,Адыгея,ТЦ,...,NaN,NaN,NaN,NaN,11.977778,NaN,NaN,12.800000,NaN,NaN
3,5,2,30,1,1,399.000000,40,offline,Адыгея,ТЦ,...,8.674419,1.0,NaN,NaN,2.500000,3.823529,8.674419,2.583333,4.050000,8.880000
4,15,2,30,1,1,169.000000,40,offline,Адыгея,ТЦ,...,2.148148,1.0,1.0,NaN,1.937500,1.333333,2.148148,2.000000,1.230769,2.142857
5,16,2,30,1,1,169.000000,40,offline,Адыгея,ТЦ,...,1.333333,1.0,1.0,1.0,1.500000,1.937500,1.333333,1.250000,2.000000,1.230769
6,34,2,30,0,0,0.000000,40,offline,Адыгея,ТЦ,...,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
7,1,2,31,4,4,699.000000,37,offline,Адыгея,ТЦ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,2,31,1,1,698.500000,37,offline,Адыгея,ТЦ,...,NaN,4.0,NaN,NaN,10.413043,NaN,NaN,11.769231,NaN,NaN
9,3,2,31,1,1,699.000000,37,offline,Адыгея,ТЦ,...,NaN,1.0,4.0,NaN,4.866667,10.413043,NaN,5.040000,11.769231,NaN


In [86]:
na_cnt = df_feat.isna().sum()
na_pct = (na_cnt / len(df_feat) * 100).round(2)
na_summary = pd.DataFrame({'NA_개수': na_cnt, 'NA_%': na_pct}).sort_values('NA_%', ascending=False)
na_summary


,NA_개수,NA_%
월별_판매건수_lag3,1030301,57.00
월별_판매량_mean3,1030301,57.00
월별_판매량_diff_lag2_3,1030301,57.00
월별_판매량_lag3,1030301,57.00
월별_평균_판매가_lag3,1030301,57.00
상품ID_도시_월별_평균_판매량_lag3,870448,48.16
월별_판매량_lag2,817631,45.23
월별_판매량_diff_lag1_2,817631,45.23
월별_판매건수_lag2,817631,45.23
월별_평균_판매가_lag2,817631,45.23


In [87]:
# 1. 데이터에 존재하는 고유한 월(날짜) 중 가장 빠른 3개를 찾습니다.
months_to_remove = sorted(df_feat['월날짜'].unique())[:3]
print(f"삭제할 월(날짜): {months_to_remove}")

# 2. 해당 3개월에 속하는 데이터를 제외합니다.
df_feat = df_feat[~df_feat['월날짜'].isin(months_to_remove)]

numeric_cols = df_feat.select_dtypes(include=np.number).columns
df_feat[numeric_cols] = df_feat[numeric_cols].fillna(0)

# 2. 범주형(category) 열만 선택해서 '없음'과 같은 적절한 문자로 채웁니다.
#    만약 '없음' 카테고리가 미리 존재하지 않았다면 추가해줘야 합니다.
category_cols = df_feat.select_dtypes(include='category').columns
for col in category_cols:
    # '없음' 이라는 카테고리가 없는 경우에만 추가
    if '없음' not in df_feat[col].cat.categories:
        df_feat[col] = df_feat[col].cat.add_categories(['없음'])
    
    # '없음'으로 NA 값을 채웁니다.
    df_feat[col] = df_feat[col].fillna('없음')


df_feat.head(30)

삭제할 월(날짜): [Timestamp('2013-01-01 00:00:00'), Timestamp('2013-02-01 00:00:00'), Timestamp('2013-03-01 00:00:00')]


,월ID,상점ID,상품ID,월별 판매량,월별 판매건수,월별 평균 판매가,상품분류ID,채널,도시,쇼핑몰 종류,...,상품ID_월별_평균_판매량_lag3,상품ID_도시_월별_평균_판매량_lag1,상품ID_도시_월별_평균_판매량_lag2,상품ID_도시_월별_평균_판매량_lag3,상품ID_상품분류ID_월별_평균_판매량_lag1,상품ID_상품분류ID_월별_평균_판매량_lag2,상품ID_상품분류ID_월별_평균_판매량_lag3,상품ID_쇼핑몰_종류_월별_평균_판매량_lag1,상품ID_쇼핑몰_종류_월별_평균_판매량_lag2,상품ID_쇼핑몰_종류_월별_평균_판매량_lag3
1,17,2,27,1,1,498.000000,19,offline,Адыгея,ТЦ,...,1.000000,1.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,5,2,30,1,1,399.000000,40,offline,Адыгея,ТЦ,...,8.674419,1.0,0.0,0.0,2.500000,3.823529,8.674419,2.583333,4.050000,8.880000
4,15,2,30,1,1,169.000000,40,offline,Адыгея,ТЦ,...,2.148148,1.0,1.0,0.0,1.937500,1.333333,2.148148,2.000000,1.230769,2.142857
5,16,2,30,1,1,169.000000,40,offline,Адыгея,ТЦ,...,1.333333,1.0,1.0,1.0,1.500000,1.937500,1.333333,1.250000,2.000000,1.230769
6,34,2,30,0,0,0.000000,40,offline,Адыгея,ТЦ,...,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9,3,2,31,1,1,699.000000,37,offline,Адыгея,ТЦ,...,0.000000,1.0,4.0,0.0,4.866667,10.413043,0.000000,5.040000,11.769231,0.000000
10,16,2,31,1,1,415.920013,37,offline,Адыгея,ТЦ,...,1.166667,1.0,1.0,4.0,1.312500,1.222222,1.166667,1.222222,1.333333,1.285714
11,33,2,31,1,1,399.000000,37,offline,Адыгея,ТЦ,...,1.000000,1.0,1.0,1.0,1.000000,1.962963,1.000000,1.000000,1.818182,1.000000
12,34,2,31,0,0,0.000000,37,offline,Адыгея,ТЦ,...,1.962963,1.0,1.0,1.0,1.285714,1.000000,1.962963,1.142857,1.000000,1.818182
13,12,2,32,1,1,119.000000,40,offline,Адыгея,ТЦ,...,2.230769,0.0,0.0,0.0,2.870968,2.531250,2.230769,3.470588,2.666667,2.555556


In [88]:
df_feat.drop(columns=['월별 판매건수', '월별 평균 판매가'], inplace=True)

In [89]:
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1621812 entries, 1 to 1807530
Data columns (total 42 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   월ID                         1621812 non-null  int8          
 1   상점ID                        1621812 non-null  int8          
 2   상품ID                        1621812 non-null  int16         
 3   월별 판매량                      1621812 non-null  int8          
 4   상품분류ID                      1621812 non-null  int8          
 5   채널                          1621812 non-null  category      
 6   도시                          1621812 non-null  category      
 7   쇼핑몰 종류                      1621812 non-null  category      
 8   상품대분류                       1621812 non-null  category      
 9   상품소분류                       1621812 non-null  category      
 10  상품_경과개월                     1621812 non-null  Int16         
 11  상품_신상여부                     1

In [90]:
df_feat = cat_encoding(df_feat, ['채널', '도시', '쇼핑몰 종류',
       '상품대분류', '상품소분류'])

Mapping for 채널: {'field': np.int64(0), 'offline': np.int64(1), 'online': np.int64(2), 'warehouse_online': np.int64(3)}
Mapping for 도시: {'Field Sale': np.int64(0), 'Online': np.int64(1), 'Адыгея': np.int64(2), 'Балашиха': np.int64(3), 'Волжский': np.int64(4), 'Вологда': np.int64(5), 'Воронеж': np.int64(6), 'Жуковский': np.int64(7), 'Казань': np.int64(8), 'Калуга': np.int64(9), 'Коломна': np.int64(10), 'Красноярск': np.int64(11), 'Курск': np.int64(12), 'Москва': np.int64(13), 'Мытищи': np.int64(14), 'Н.Новгород': np.int64(15), 'Новосибирск': np.int64(16), 'Омск': np.int64(17), 'РостовНаДону': np.int64(18), 'СПб': np.int64(19), 'Самара': np.int64(20), 'Сергиев Посад': np.int64(21), 'Сургут': np.int64(22), 'Томск': np.int64(23), 'Тюмень': np.int64(24), 'Уфа': np.int64(25), 'Химки': np.int64(26), 'Чехов': np.int64(27), 'Якутск': np.int64(28), 'Якутск Орджоникидзе, 56': np.int64(29), 'Якутск Орджоникидзе, 56 фран': np.int64(30), 'Ярославль': np.int64(31)}
Mapping for 쇼핑몰 종류: {'etc': np.int64

In [91]:
df_feat.to_csv('data/output/20250908_modeling_data.csv', index=False)

In [92]:
# # ===== 4) 인사이트용 시간 파생피처 =====
# df_feat['month'] = df_feat['월날짜'].dt.month
# df_feat['month_sin'] = np.sin(2*np.pi*df_feat['month']/12.0)
# df_feat['month_cos'] = np.cos(2*np.pi*df_feat['month']/12.0)

# df_feat['quarter'] = df_feat['월날짜'].dt.quarter
# df_feat['quarter_sin'] = np.sin(2*np.pi*df_feat['quarter']/4.0)
# df_feat['quarter_cos'] = np.cos(2*np.pi*df_feat['quarter']/4.0)

# df_feat['_ym'] = df_feat['월날짜'].dt.year*12 + df_feat['월날짜'].dt.month
# min_ym = df_feat['_ym'].min()
# df_feat['t_idx'] = df_feat['_ym'] - min_ym

# sale = df_feat['월별 판매량'].fillna(0)

# # 상품 레벨 수명
# tmp = np.where(sale>0, df_feat['_ym'], np.nan)
# first_sale_prod = pd.Series(tmp, index=df_feat.index).groupby(df_feat['상품ID']).transform('min')
# df_feat['since_first_sale_prod_m'] = df_feat['_ym'] - first_sale_prod

# # 매장×상품 레벨 수명
# tmp = np.where(sale>0, df_feat['_ym'], np.nan)
# first_sale_store_item = pd.Series(tmp, index=df_feat.index).groupby([df_feat['상점ID'], df_feat['상품ID']]).transform('min')
# df_feat['since_first_sale_store_item_m'] = df_feat['_ym'] - first_sale_store_item

# # 상품별 활성 매장 수(+ lag)
# active_store = (
#     df_feat.assign(sold=(df_feat['월별 판매량']>0).astype(int))
#           .groupby(['상품ID','월날짜'], observed=True)
#           .agg(active_store_cnt=('상점ID','nunique'))
#           .reset_index()
#           .sort_values(['상품ID','월날짜'])
# )
# active_store['active_store_cnt_lag1'] = active_store.groupby('상품ID')['active_store_cnt'].shift(1)
# active_store['active_store_cnt_lag3'] = active_store.groupby('상품ID')['active_store_cnt'].shift(3)

# df_feat = df_feat.merge(
#     active_store[['상품ID','월날짜','active_store_cnt','active_store_cnt_lag1','active_store_cnt_lag3']],
#     on=['상품ID','월날짜'], how='left'
# )

# # 연속 제로런 & 최근 3개월 판매여부
# is_zero = (df_feat.groupby(['상점ID','상품ID'], observed=True)['월별 판매량']
#            .transform(lambda s: (s==0).astype(int)))
# is_zero_prev = is_zero.groupby([df_feat['상점ID'], df_feat['상품ID']], observed=True).shift(1, fill_value=0)
# block_id = (is_zero_prev==0).groupby([df_feat['상점ID'], df_feat['상품ID']], observed=True).cumsum()
# df_feat['consec_zero_run_prev'] = is_zero_prev.groupby(block_id, observed=True).cumsum()

# sold_flag = (df_feat['월별 판매량']>0).astype(int)
# sold_flag = sold_flag.groupby([df_feat['상점ID'], df_feat['상품ID']], observed=True).shift(1, fill_value=0)
# df_feat['sold_in_last3m'] = (sold_flag.groupby([df_feat['상점ID'], df_feat['상품ID']], observed=True)
#                              .rolling(3, min_periods=1).sum()
#                              .reset_index(level=[0,1], drop=True) > 0).astype(int)

# # 내부 계산용 컬럼 정리
# df_feat.drop(columns=['_ym'], inplace=True)